In [ ]:
# elevation gain calculation attempt
import requests
import urllib
import pandas as pd

# USGS Elevation Point Query Service
#url = r'https://nationalmap.gov/epqs/pqs.php?'
#new 2023:
url = r'https://epqs.nationalmap.gov/v1/json?'

# coordinates with known elevation 
lat = [48.633, 48.733, 45.1947, 45.1962]
lon = [-93.9667, -94.6167, -93.3257, -93.2755]
   
# create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': lon
})

def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):
                
        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }
        
        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        #elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])
        #new 2023:
        elevations.append(result.json()['value'])

    df['elev_meters'] = elevations

elevation_function(df, 'lat', 'lon')
df.head()
    

In [ ]:
url = r'https://epqs.nationalmap.gov/v1/json?'
params = {
            'output': 'json',
            'x':-93.9667 ,
            'y': 48.633,
            'units': 'Meters'
    }
result = requests.get((url + urllib.parse.urlencode(params)))
float(result.json()['value'])

In [ ]:
def elevation_gain(lat1,lon1,lat2,lon2):
    url = r'https://epqs.nationalmap.gov/v1/json?'

    # Start location
    params = {
            'output': 'json',
            'x':lon1 ,
            'y': lat1,
            'units': 'Meters'
    }
    result = requests.get((url + urllib.parse.urlencode(params)))
    elevation_start = float(result.json()['value'])
    

    # End location
    params = {
            'output': 'json',
            'x':lon2 ,
            'y': lat2,
            'units': 'Meters'
    }

    result = requests.get((url + urllib.parse.urlencode(params)))
    elevation_end = float(result.json()['value'])

    if (elevation_end-elevation_start)<=0:
        elev_gain = 0
        print(elev_gain)
    else:
        elev_gain = elevation_end-elevation_start 
        print(elev_gain)
    
    return elev_gain
 

In [ ]:
from pyomo.environ import units as pyunits

In [ ]:
N = 3
capacity = 1000
elev_gain = 50*pyunits.m #m
pumping_velocity = 1.5 *pyunits.m/pyunits.s
storage_capacity = capacity* pyunits.m**3/pyunits.day
pipe_csa = pyunits.convert(storage_capacity,to_units = pyunits.m**3/pyunits.s)/pumping_velocity
pump_eff = 0.9
motor_eff = 0.9
lift_height = pyunits.convert(elev_gain,to_units=pyunits.ft)
flow = pumping_velocity* pipe_csa
flow_in_gpm  =flow*(pyunits.convert(1*pyunits.m**3/pyunits.s, to_units=(pyunits.gallons / pyunits.minute)))
pump_power_kw = (0.746 * flow_in_gpm * lift_height / (3960 * pump_eff * motor_eff)) * pyunits.kilowatts
electricity = (N+1)*pump_power_kw()
print(electricity)

In [ ]:
friction_factor = 0.005
density = 1000*pyunits.kg/pyunits.m**3
g = 9.8*pyunits.m/pyunits.s**2
pipe_diameter = 2 * pyunits.inch    
deltaP_grad =  friction_factor*density*(pumping_velocity**2)/(2*pyunits.convert(pipe_diameter,to_units = pyunits.m))*1e-5 #bar/m
deltaP_grad()

In [ ]:
elev_gain = 2000*pyunits.m #m
distance = 100 *pyunits.km
deltaP_elev_gain = density*g*elev_gain*pyunits.m/pyunits.convert(distance,pyunits.m) * 1e-5
deltaP_elev_gain()